---
title: "Notebook to establish mini-batch training from first principles"
author: "John Richmond"
date: "November 2nd, 2022"
toc: true
format:
  html:
    code-fold: False
    code-tools: true
    number-sections" false
  pdf:
    geometry:
      - top=30mm
      - left=20mm

## Notebook to explore Python fundamentals relevant to Deep Learning and Data Science

In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI events - demonstration of the principal using widgets

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [18]:
import ipywidgets as widgets
from time import sleep

In [13]:
btn = widgets.Button(description="Click Me")

Define an action function to carry out when the button is clicked

In [14]:
def btn_action(widget):
    print('Button pressed')

Now assign the function to the putton pressed callback for the widget

In [15]:
btn.on_click(btn_action)

In [16]:
btn

Button(description='Click Me', style=ButtonStyle())

Button pressed
Button pressed
Button pressed


### Creating your own callbacks 

Callbacks can be implemented in any function as per the following

In [17]:
def show_progress_cb(progress):
    print(f"Finished epoch: {progress}")

In [55]:
def dummy_training_loop(cb=None):
    epoch=0
    for i in range(10):
        # Dummy model run
        sleep(0.25)
        if cb: cb(epoch)
        epoch += 1
    

In [25]:
dummy_training_loop(show_progress_cb)

Finished epoch: 0
Finished epoch: 1
Finished epoch: 2
Finished epoch: 3
Finished epoch: 4
Finished epoch: 5
Finished epoch: 6
Finished epoch: 7
Finished epoch: 8
Finished epoch: 9


### Lambdas and partials 

Lambdas are effectively one line functions

In [27]:
dummy_training_loop(lambda o: print(f"Finished epoch: {o}"))

Finished epoch: 0
Finished epoch: 1
Finished epoch: 2
Finished epoch: 3
Finished epoch: 4
Finished epoch: 5
Finished epoch: 6
Finished epoch: 7
Finished epoch: 8
Finished epoch: 9


Using a partial to add to the capability

In [41]:
def show_progress_cb(progress, exclamation):
    print(f"({exclamation}, finished epoch: {progress}")

In [34]:
from functools import partial

In [44]:
# Note that it doesn't seem possible to use the keywork for the first argument of the function for some 
# reason, instead you just have to put the actual value.  So if exclamation was the first argument you 
# would have to use
#f2 = partial(show_progress_cb, "Nice")
f2 = partial(show_progress_cb, exclamation="Nice")

In [45]:
dummy_training_loop(f2)

(Nice, finished epoch: 0
(Nice, finished epoch: 1
(Nice, finished epoch: 2
(Nice, finished epoch: 3
(Nice, finished epoch: 4
(Nice, finished epoch: 5
(Nice, finished epoch: 6
(Nice, finished epoch: 7
(Nice, finished epoch: 8
(Nice, finished epoch: 9


### Callbacks as callable classes 

In [49]:
class PrintProgressCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation
        
    def __call__(self, progress):
        print(f"{self.exclamation}, finished epoch: {progress}")

In [50]:
cb = PrintProgressCallback("Just magic")

In [51]:
dummy_training_loop(cb)

Just magic, finished epoch: 0
Just magic, finished epoch: 1
Just magic, finished epoch: 2
Just magic, finished epoch: 3
Just magic, finished epoch: 4
Just magic, finished epoch: 5
Just magic, finished epoch: 6
Just magic, finished epoch: 7
Just magic, finished epoch: 8
Just magic, finished epoch: 9


### Expanding this to multiple callback functions using *args and **kwargs

In [52]:
def f(*a, **b): print(f"args: {a}; kwargs: {b}")

In [53]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [54]:
def g(a,b,c=0): print(a,b,c)
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


In the above look lets have two callbacks, one prior to the dummy model run and one after

In [74]:
def dummy_training_loop(cb=None):
    epoch=0
    for i in range(5):
        if cb: cb.pre_calc(epoch)
        # Dummy model run
        sleep(0.25)
        res = i*i
        if cb: cb.post_calc(epoch, res)
        epoch += 1

In [78]:
class PrintProgressCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation
        
    def pre_calc(self, progress, *args, **kwargs):
        print(f"Starting epoch: {progress}")
        
    def post_calc(self, progress, acc, *args, **kwargs):
        print(f"{self.exclamation}, finished epoch: {progress}, accuracy: {acc}")    

In [76]:
cb = PrintProgressCallback("Great")

In [77]:
dummy_training_loop(cb)

Starting epoch: 0
Great, finished epoch: 0, accuracy: 0
Starting epoch: 1
Great, finished epoch: 1, accuracy: 1
Starting epoch: 2
Great, finished epoch: 2, accuracy: 4
Starting epoch: 3
Great, finished epoch: 3, accuracy: 9
Starting epoch: 4
Great, finished epoch: 4, accuracy: 16


The above solution can be made more robust by using the hasattr function to check that the callback exists.  We can also make the callback modify behaviour, for example if a variable exceeds a limit


In [79]:
def dummy_training_loop(cb=None):
    epoch=0
    for i in range(5):
        if cb and hasattr(cb, 'pre_calc'): cb.pre_calc(epoch)
        # Dummy model run
        sleep(0.25)
        res = i*i
        if cb and hasattr(cb, 'post_calc'):
            if cb.post_calc(epoch, res):
                print("Early stopping triggered")
                break
        epoch += 1

In [80]:
class PrintProgressCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation
        
    def pre_calc(self, progress, *args, **kwargs):
        print(f"Starting epoch: {progress}")
        
    def post_calc(self, progress, acc, *args, **kwargs):
        print(f"{self.exclamation}, finished epoch: {progress}, accuracy: {acc}")
        if acc > 5: return True

In [81]:
dummy_training_loop(PrintProgressCallback())

Starting epoch: 0
Awesome, finished epoch: 0, accuracy: 0
Starting epoch: 1
Awesome, finished epoch: 1, accuracy: 1
Starting epoch: 2
Awesome, finished epoch: 2, accuracy: 4
Starting epoch: 3
Awesome, finished epoch: 3, accuracy: 9
Early stopping triggered


This can be updated again so that the callback can change variables in the calling class by passing the class itself to the callback as follows

In [154]:
class ModelTrainingDummy():
    def __init__(self, cb=None):
        self.cb, self.res = cb, 0
        
    def callback(self, cb_name, *args):
        # The the callback does not exist then return
        if not self.cb: return
        
        # get the callback based upon the name
        cb = getattr(self.cb, cb_name, None)
        pdb: set_trace()
        if cb: return cb(self, *args)
    
    def calc(self):
        for epoch in range(6):
            self.callback('before_calc', epoch)
            self.res += epoch * epoch
            sleep(0.25)
            if self.callback("after_calc", epoch):
                # this point is reached if the callback return True
                print('early stopping')
                break

In [155]:
class ModifyingCallback():
    def __init__(self):
        pass
    
    def before_calc(self, calc, val):
        print (f"starting epoch {val}: res={calc.res}")
        
    def after_calc(self, calc, val):
        if val>4:
            return True
        else:
            if calc.res <3:
                print(f"original res: {calc.res}")
                calc.res = calc.res*2
                print(f"modified res: {calc.res}")
                

In [156]:
trainer = ModelTrainingDummy(ModifyingCallback())

In [157]:
trainer.calc()

starting epoch 0: res=0
original res: 0
modified res: 0
starting epoch 1: res=0
original res: 1
modified res: 2
starting epoch 2: res=2
starting epoch 3: res=6
starting epoch 4: res=15
starting epoch 5: res=31
early stopping


It can be seen that as well as the early stopping the callback is modifying the properties of the calling class, which is possible since it class is sent to the callback as an argument

### __dunder__ methods

Anything that looks like `__this__` is, in some way, *special*. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call `__init__`. These are defined as part of the python [data model](https://docs.python.org/3/reference/datamodel.html#object.__init__).

For instance, if python sees `+`, then it will call the special method `__add__`. If you try to display an object in Jupyter (or lots of other places in Python) it will call `__repr__`.

In [163]:
class DemoDunder():
    def __init__(self, val): self.val = val
    def __add__(self, b):
        return(self.val + b.val + 0.01)
    def __repr__(self):
        return str(self.val)
        

In [164]:
a = DemoDunder(5)
b = DemoDunder(6)
a+b

11.01

Special methods to know about include (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

In [165]:
class A: a,b=1,2

In [166]:
a=A()

In [169]:
getattr(a, 'b') 

2

In [170]:
getattr(a, 'a')

1

Note that the getattr call only occurs if the properly cannot be found normally (ie if does not exist),  This allows it to catch errors or to implement things based upon an attribute name that has not been implemented

In [186]:
class B():
    a, b, _c = 1, 2, 3
    def __getattr__(self, k):
        print(k)
        if k[0] == '_': raise Exception(f"invalid attribute {k}")
        return f"Attribute has value: {k}"
            

In [187]:
b=B()
b.a

1

In [190]:
b.d

d


'Attribute has value: d'

In [191]:
b._e

_e


Exception: invalid attribute _e